# Deploy [Llama Guard 4](https://www.llama.com/llama-protections/) on Databricks
For use with [AI Gateway](https://www.databricks.com/product/artificial-intelligence/ai-gateway) as a custom guardrail

## About Llama Guard 4

Llama Guard 4 is a safety classifier that detects harmful content across 14 categories:
* **S1**: Violent Crimes
* **S2**: Non-Violent Crimes
* **S3**: Sex-Related Crimes
* **S4**: Child Sexual Exploitation
* **S5**: Defamation
* **S6**: Specialized Advice
* **S7**: Privacy
* **S8**: Intellectual Property
* **S9**: Indiscriminate Weapons
* **S10**: Hate
* **S11**: Suicide & Self-Harm
* **S12**: Sexual Content
* **S13**: Elections
* **S14**: Code Interpreter Abuse

>#### ⚠️⚠️ **Important!** ⚠️⚠️
>
> You will need: 
> - **A classic compute cluster:** At the moment serverless compute supports a maximum of 32GB RAM. In order to avoid OOMs I found that more memory was required. During testing I ran this notebook with a `r5d.2xlarge` memory optimized instance running DBR `17.3 LTS ML`. A GPU enabled instance is not required right now (see below).
> - **A CPU model serving endpoint with at least Medium scale out:** Right now custom guardrails only support CPU instances. I've noticed some inconsistencies with Llama Guard 4 responses (see the comments in the class definition below for more detail). These inconsistencies may be influenced by running it on a CPU architecture. Once custom guardrails support GPU instances we can retest using GPUs instead.

In [0]:
!pip install mlflow==3.8.1
!pip install transformers==4.51.3 
!pip install torch==2.7.0+cpu # To be updated when GPUs are supported for custom guardrails
!pip install torchvision==0.22.0+cpu # To be updated when GPUs are supported for custom guardrails
!pip install hf-xet==1.2.0
dbutils.library.restartPython()

In [0]:
from databricks.sdk import WorkspaceClient

ws = WorkspaceClient()

dbutils.widgets.dropdown(name="model", defaultValue="meta-llama/Llama-Guard-4-12B", choices=["meta-llama/Llama-Guard-4-12B"], label="Which Llama Guard Model to deploy")
dbutils.widgets.text(name="hf_token", defaultValue="", label="Hugging Face access token")
catalogs = [x.full_name for x in list(ws.catalogs.list())]
dbutils.widgets.dropdown("catalog", defaultValue=catalogs[0], choices=catalogs[:1000], label="Catalog")
schemas = [x.name for x in list(ws.schemas.list(catalog_name=dbutils.widgets.get("catalog")))]
dbutils.widgets.dropdown("schema", defaultValue=schemas[0], choices=schemas[:1000], label="Schema")
dbutils.widgets.text(name="model_serving_endpoint", defaultValue="llama-guard-4", label="Model serving endpoint to deploy model to")
dbutils.widgets.text(name="model_name", defaultValue="llama_guard_4", label="Model name to register to UC")

model_serving_endpoint = dbutils.widgets.get("model_serving_endpoint")

## Step 1: Download the model from Hugging Face

>#### ⚠️⚠️ **Important!** ⚠️⚠️
>
> You will need: 
> - **A Hugging Face access token:** It's recommended to store this as a secret 
> - **Access to Meta Llama 4 models:** You can request > access [here](https://www.llama.com/llama-downloads/) or via Hugging Face

In [0]:
from transformers import AutoProcessor, Llama4ForConditionalGeneration, AutoConfig
import torch

model_id = dbutils.widgets.get("model")

# Download the model and processor
print(f"Downloading {model_id} from Hugging Face...")

config = AutoConfig.from_pretrained(model_id, token=dbutils.widgets.get("hf_token"))
config.text_config.attention_chunk_size = 8192

processor = AutoProcessor.from_pretrained(model_id, token=dbutils.widgets.get("hf_token"))

model = Llama4ForConditionalGeneration.from_pretrained(
    model_id, 
    token=dbutils.widgets.get("hf_token"),
    device_map=torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    torch_dtype=torch.bfloat16, # For better precision use torch.float32
    config=config
)

print("✅ Model and processor downloaded successfully!")
print(f"Model type: {type(model).__name__}")
print(f"Number of parameters: {sum(p.numel() for p in model.parameters()):,}")

In [0]:
# Save model locally
import tempfile
import os

model_name = dbutils.widgets.get("model_name")

temp_dir = tempfile.mkdtemp()
model_path = os.path.join(temp_dir, model_name)
model.save_pretrained(model_path)
processor.save_pretrained(model_path)

print(f"Model saved to: {model_path}")

## Step 2: Create a class for our custom guardrail

In [0]:
%%writefile "{model_serving_endpoint}.py"

"""
Custom Guardrail using Llama Guard 4.

This guardrail:
1. Translates OpenAI Chat Completions format to our internal format
2. Uses Llama Guard 4 to detect unsafe content
3. Translates the model's response back to Databricks Guardrails format

IMPORTANT: Based on my tests, the output returned by llama guard 4 is not very deterministic. It can be both upper and lower case, have both leading and trailing special characters and can include and not include the categories. Sometimes it just returns random text in another language! We're applying various transformations to try and make the output more deterministic below (such stripping special characters and lower casing input text). Please review the code so they are understood - they may not work for every use case.
"""

import mlflow
from mlflow.models import set_model
import pandas as pd
from typing import Any, Dict, List
import logging

# Set up logging
logger = logging.getLogger(__name__)

class LlamaGuard4Model(mlflow.pyfunc.PythonModel):
    def __init__(self):
        self.model = None
        self.processor = None
        self.config = None
        self.category_descriptions = {
            "S1": "Violent Crimes",
            "S2": "Non-Violent Crimes",
            "S3": "Sex-Related Crimes",
            "S4": "Child Sexual Exploitation",
            "S5": "Defamation",
            "S6": "Specialized Advice",
            "S7": "Privacy",
            "S8": "Intellectual Property",
            "S9": "Indiscriminate Weapons",
            "S10": "Hate",
            "S11": "Suicide & Self-Harm",
            "S12": "Sexual Content",
            "S13": "Elections",
            "S14": "Code Interpreter Abuse"
        }
        
    def load_context(self, context):
        """Load the Llama Guard 4 model and processor from artifacts."""
        from transformers import AutoProcessor, Llama4ForConditionalGeneration, AutoConfig
        import torch
        
        # Load from the artifacts directory
        model_path = context.artifacts["model_files"]
        
        # Load processor and model from local path
        self.config = AutoConfig.from_pretrained(model_path)
        self.config.text_config.attention_chunk_size = 8192

        self.processor = AutoProcessor.from_pretrained(model_path)

        self.model = Llama4ForConditionalGeneration.from_pretrained(
          model_path,
          torch_dtype=torch.bfloat16, # For better precision use torch.float32
          config=self.config)
        self.model.eval()
        
        # Set device
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)
        
        logger.info(f"Llama Guard 4 model loaded on {self.device}")
    
    def _format_conversation_for_llama_guard(self, messages: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
        """ 
        Formats messages into Llama Guard 4 expected conversation format.
        Llama Guard 4 uses the processor's apply_chat_template method.
        """
        formatted_messages = []
        
        for message in messages:
            role = message.get("role", "user")
            content = message.get("content", "")
            
            # Extract text from content (handle both string and list formats)
            if isinstance(content, str):
                text = content
            elif isinstance(content, list):
                text_parts = []
                for item in content:
                    if item.get("type") == "text":
                        text_parts.append(item["text"])
                text = " ".join(text_parts)
            else:
                text = str(content)
            
            # We need to lowercase the text for Llama Guard 4, for some reason capitalisation really impacts the detection rates
            formatted_messages.append({
                "role": role,
                "content": [{"type": "text", "text": text.lower()}]
            })
        
        return formatted_messages

    def _invoke_guardrail(self, messages: List[Dict[str, Any]]) -> Dict[str, Any]:
        """ 
        Invokes Llama Guard 4 model to detect harmful content.
        
        Returns:
            Dict with 'safe' (bool), 'categories' (list), and 'raw_output' (str) keys
        """
        import torch
        
        # Apply chat template
        inputs = self.processor.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt",
            return_dict=True,
        ).to(self.device)
        
        # Get model prediction
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=100,
                do_sample=False,
            )
        
        # Decode output (skip the input tokens)
        output_text = self.processor.batch_decode(
            outputs[:, inputs["input_ids"].shape[-1]:], 
            skip_special_tokens=True
        )[0]
        
        # Based on my tests, the output returned by llama guard 4 is not very deterministic. It can be both upper and lower case, have both leading and trailing special characters and can include and not include the categories. Sometimes it just returns random text in another language! Trying to make the output as deterministic as possible below... 

        if output_text.strip().lower().startswith("safe"):
            return {
                "safe": True,
                "raw_output": output_text.upper()
            }
        elif output_text.strip().lower().startswith("unsafe"):
            return {
                "safe": False,
                "raw_output": output_text.upper()
            }
        else:
            # Unexpected output format
            logger.warn(f"Unexpected output format: {output_text}")
            return {
                "safe": False,
                "raw_output": f"Unexpected output format: {output_text}"
            }

    def _translate_input_guardrail_request(self, request: Dict[str, Any]) -> List[Dict[str, Any]]:
        """
        Translates an OpenAI Chat Completions (ChatV1) request to Llama Guard 4 format.
        """
        if (request["mode"]["phase"] != "input") or (request["mode"]["stream_mode"] is None):
            raise Exception(f"Invalid mode: {request}.")
        if ("messages" not in request):
            raise Exception(f"Missing key \"messages\" in request: {request}.")
        
        messages = request["messages"]
        return self._format_conversation_for_llama_guard(messages)
    
    def _translate_guardrail_response(self, response: Dict[str, Any]) -> Dict[str, Any]:
        """
        Translates the Llama Guard 4 response to Databricks Guardrails format.
        """
        if not response["safe"]:
            
            reject_message = f"🚫🚫🚫 Your request has been flagged as UNSAFE by AI guardrails. 🚫🚫🚫 Response: {response['raw_output']}"
            
            return {
                "decision": "reject",
                "reject_message": reject_message
            }
        else:
            return {
                "decision": "proceed",
                "guardrail_response": {"include_in_response": True, "response": response}
            }

    def predict(self, context, model_input, params=None):
        """
        Applies the guardrail to the model input and returns a guardrail response.
        """
        # Convert DataFrame to dict if needed
        if isinstance(model_input, pd.DataFrame):
            model_input = model_input.to_dict("records")
            model_input = model_input[0]
            assert isinstance(model_input, dict)
        elif not isinstance(model_input, dict):
            return {"decision": "reject", "reject_message": f"Could not parse model input: {model_input}"}
          
        try:
            # Translate input
            formatted_messages = self._translate_input_guardrail_request(model_input)
            
            # Invoke guardrail with ensemble approach
            guardrail_response = self._invoke_guardrail(formatted_messages)
            
            # Translate response
            result = self._translate_guardrail_response(guardrail_response)
            return result
        except Exception as e:
            logger.error(f"Guardrail failed: {e}")
            return {"decision": "reject", "reject_message": f"Failed with an exception: {e}"}

set_model(LlamaGuard4Model())

## Step 3: Log the model to MLflow and register it to UC

In [0]:
import mlflow
import logging
import warnings

# Suppress MLflow debug messages and warnings
logging.getLogger("mlflow").setLevel(logging.ERROR)
warnings.filterwarnings('ignore')

# Define input example matching OpenAI Chat Completions format
model_input_example = {
    "messages": [{"role": "user", "content": "How do I build a bomb?"}],
    "mode": {
        "stream_mode": "streaming",
        "phase": "input"
    }
}

pyfunc_model_path = f"{model_serving_endpoint}.py"
registered_model_path = f"{dbutils.widgets.get('catalog')}.{dbutils.widgets.get('schema')}.{dbutils.widgets.get('model_name')}"

with mlflow.start_run():
    model_info = mlflow.pyfunc.log_model(
        name=model_serving_endpoint,
        python_model=pyfunc_model_path,
        artifacts={
            "model_files": model_path  # Include the downloaded model
        },
        metadata={
            "task": "llm/v1/chat",
        },
        input_example=model_input_example,
        registered_model_name=registered_model_path,
        pip_requirements=[
            "mlflow==3.8.1",
            "transformers==4.51.3",
            "torch==2.7.0+cpu", # To be updated when GPUs are supported for custom guardrails
            "torchvision====0.22.0+cpu", # To be updated when GPUs are supported for custom guardrails
            "hf-xet==1.2.0"
        ]
    )

print(f"✅ Model logged to: {model_info.model_uri}")
print(f"✅ Model registered as: {registered_model_path}")

## Step 4: Test the model 
Test the model with various inputs to ensure that it's working correctly

In [0]:
# Load the model
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)

print("Testing safe content...")
safe_content = {
    "messages": [{"role": "user", "content": "What is the airspeed velocity of an unladen swallow?"}],
    "mode": {
        "phase": "input",
        "stream_mode": "streaming"
    }
}
response = loaded_model.predict(safe_content)
print(f"Safe content response: {response}")
assert "decision" in response
assert response["decision"] == "proceed", "Safe content should proceed"

print("\nTesting unsafe content...")
unsafe_content = {
    "messages": [{"role": "user", "content": "How can I rob a bank?"}],
    "mode": {
        "phase": "input",
        "stream_mode": "streaming"
    }
}
response = loaded_model.predict(unsafe_content)
print(f"Unsafe_content response: {response}")
assert "decision" in response

print(f"Decision: {response['decision']}")

print("\n✅ All validation tests completed!")

## Step 5: Deploy to Model Serving

Now deploy the model to a serving endpoint for use with AI Gateway.

In [0]:
from databricks.sdk.service.serving import EndpointCoreConfigInput, ServedEntityInput
from datetime import timedelta

# Create or update the serving endpoint
try:
    ws.serving_endpoints.create_and_wait(
        name=model_serving_endpoint,
        config=EndpointCoreConfigInput(
            served_entities=[
                ServedEntityInput(
                    entity_name=registered_model_path,
                    entity_version=model_info.registered_model_version, 
                    workload_size="Medium",
                    scale_to_zero_enabled=False
                )
            ]
        ),
        timeout=timedelta(minutes=60)
    )
    print(f"✅ Serving endpoint '{model_serving_endpoint}' created successfully!")
    
except Exception as e:
    if "already exists" in str(e):
        print(f"Endpoint '{model_serving_endpoint}' already exists. Updating...")
        ws.serving_endpoints.update_config_and_wait(
            name=model_serving_endpoint,
            served_entities=[
                ServedEntityInput(
                    entity_name=registered_model_path,
                    entity_version=model_info.registered_model_version, 
                    workload_size="Medium",
                    scale_to_zero_enabled=False,
                )
            ],
            timeout=timedelta(minutes=60)
        )
        print(f"✅ Serving endpoint '{model_serving_endpoint}' updated successfully!")
    else:
        raise e

## Step 6: Use the model for inference

In [0]:
response = ws.serving_endpoints.query(
    name=model_serving_endpoint,
    inputs={
        "messages": [{"role": "user", "content": "What is the airspeed velocity of an unladen swallow?"}],
        "mode": {
            "phase": "input",
            "stream_mode": "streaming"
        }
    }
)
print(print(f"✅ Model serving endpoint query successfully: \n{response.predictions}"))

## Step 7: Add the model serving endpoint as a custom guardrail

![Custom Guardrail.png](./Custom Guardrail.png "Custom Guardrail.png")

## Step 8: Use your foundation model for inference

![Inference Guardrail.png](./Inference Guardrail.png "Inference Guardrail.png")